In [4]:
import mysql.connector as mysql
import json
import os
import io
import csv
import random
from collections import OrderedDict
from datetime import datetime
from wrapper import Database, data_inspector

with open('config.json', 'r') as read_file:
    client = json.load(read_file)

In [13]:
path = 'data\\Sharif University of Technology'
files = list(os.walk(path))[0][2]

with io.open(os.path.join(path, files[0]), 'r', encoding='utf8') as raw:
    data = json.load(raw)

faculties = []
with io.open('data\\faculties.csv', 'r', encoding='utf-8-sig') as csvFile:
    reader = csv.DictReader(csvFile)
    for row in reader:
        if row['Scopus']:
            row['Scopus'] = list(map(int, row['Scopus'].split(',')))
        faculties.append(row)

asjc = []
with io.open('data\\ASJC Codes.csv', 'r', encoding='utf-8-sig') as csvFile:
    reader = csv.DictReader(csvFile)
    for row in reader:
        asjc.append(row)

sources = []
with io.open('data\\Scopus Sources.csv', 'r', encoding='utf-8-sig') as csvFile:
    reader = csv.DictReader(csvFile)
    to_be_removed = [
        'Active', 'Discontinued', 'Coverage', '2016 CiteScore', '2017 CiteScore', '2018 CiteScore',
        'Medline-sourced', 'Open Access', 'Articles in Press Included', 'Added to list April 2019',
        'Title history indication', 'Related title to title history indication', 'Other related title 1',
        'Other related title 2', 'Other related title 3', 'Publisher imprints grouped to main Publisher',
    ]
    for row in reader:
        for col in to_be_removed:
            row.pop(col, None)
        row['ASJC'] = [int(code)
                       for code in row['ASJC'].split(';') if code != '']
        sources.append(row)

In [ ]:
# d = Database(config = client, db_name = 'scopus')
# d._insert_many('paper_funding', [{'agency_id_scp': 27, 'agency': 'FQatar', 'agency_acronym': 'QNSF'}])#,{'agency_id_scp': 13, 'agency': 'US', 'agency_acronym': 'NSF'}])
# d._insert_many('source_subject', [{'source_id': 2, 'subject_id': 2},{'subject_id': 3, 'source_id': 1}])
# d._read('paper_funding', {'agency_id_scp': {'value': 2, 'operator': '='}}, result_columns=True)
# d._insert_one('paper_funding', {'agency_id_scp': 2, 'agency': 'FQatar', 'agency_acronym': 'QNSF'})#,{'agency_id_scp': 13, 'agency': 'US', 'agency_acronym': 'NSF'}])
# d._read('source_subject', {'source_id': {'value': 1, 'operator': '='}, 'subject_id': {'value': 3, 'operator': '='}}, result_columns=True)
# d._table_order()
# print(d.describe('subject'))
# d._close()

In [5]:
paper_cnt = 0
skipped_cnt = 0

db = Database(config=client, db_name='scopus')

path = 'data\\Sharif University of Technology'
files = list(os.walk(path))[0][2]

for file in files[:1]:
    with io.open(os.path.join(path, file), 'r', encoding='utf8') as raw:
        data = json.load(raw)
    data = data['search-results']['entry']
    ret_time = datetime.utcfromtimestamp(
        int(file.split('.')[0].split('_')[-1])).strftime('%Y-%m-%d %H:%M:%S')
    for paper in data:
        warnings = data_inspector(paper)
        print(file)
        print(paper['dc:identifier'])
        ins_id = db.raw_insert(paper, retrieval_time=ret_time)
        print(ins_id)
        paper_cnt += 1
        if not ins_id['value']:
            skipped_cnt += 1
        print('---------------------')
db._close()
print()
print()
print(f'{len(files)} file reviewed')
print(f'{paper_cnt} papers reviewed')
print(f'{skipped_cnt} papers skipped')
print(f'{paper_cnt - skipped_cnt} papers inserted into the db')
# ins_id

Sharif University of Technology_y2018_000_S9J79E_1558880307.txt
SCOPUS_ID:85059543553
{'msg': 'paper exists', 'value': 1}
---------------------
Sharif University of Technology_y2018_000_S9J79E_1558880307.txt
SCOPUS_ID:85051772463
{'msg': 'paper exists', 'value': 2}
---------------------
Sharif University of Technology_y2018_000_S9J79E_1558880307.txt
SCOPUS_ID:85063011120
{'msg': 'paper exists', 'value': 3}
---------------------
Sharif University of Technology_y2018_000_S9J79E_1558880307.txt
SCOPUS_ID:85059513265
{'msg': 'paper exists', 'value': 4}
---------------------
Sharif University of Technology_y2018_000_S9J79E_1558880307.txt
SCOPUS_ID:85060849059
{'msg': 'paper exists', 'value': 5}
---------------------
Sharif University of Technology_y2018_000_S9J79E_1558880307.txt
SCOPUS_ID:85055661422
{'msg': 'paper exists', 'value': 6}
---------------------
Sharif University of Technology_y2018_000_S9J79E_1558880307.txt
SCOPUS_ID:85059028391
{'msg': 'paper exists', 'value': 7}
--------------